In [2]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import regularizers
from matplotlib import pyplot as plt
import tensorflow as tf
from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from tf.keras.losses import sparse_categorical_crossentropy
from keras.losses import sparse_categorical_crossentropy
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D,BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications import ResNet50, MobileNet
from keras.models import Model
from sklearn.model_selection import train_test_split

In [21]:
img_height,img_width=(128,128)
batch_size=12
train_data_dir="Dataset/"

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.3
                                   )
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                target_size=(img_height,img_width),
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='training')
test_generator = train_datagen.flow_from_directory(train_data_dir,
                                                target_size=(img_height,img_width),
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='validation')

Found 14818 images belonging to 4 classes.
Found 6347 images belonging to 4 classes.


In [3]:

# (3) Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(img_height,img_width,3), kernel_size=(11,11),
 strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(img_width,img_height,3)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(4))
model.add(Activation('softmax'))

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 96)        34944     
_________________________________________________________________
activation (Activation)      (None, 30, 30, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 15, 15, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 256)       2973952   
_________________________________________________________________
activation_1 (Activation)    (None, 15, 15, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0

In [4]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

history=model.fit(train_generator,
          epochs=25,
        validation_data=test_generator)
model.save("models/alexnet.h5")

Epoch 1/25
1235/1235 [==============================] - 2036s 2s/step - loss: 1.1460 - accuracy: 0.5628 - val_loss: 1.4356 - val_accuracy: 0.5831
Epoch 2/25
1235/1235 [==============================] - 1964s 2s/step - loss: 0.9033 - accuracy: 0.6355 - val_loss: 0.9871 - val_accuracy: 0.5852
Epoch 3/25
1235/1235 [==============================] - 2014s 2s/step - loss: 0.8348 - accuracy: 0.6680 - val_loss: 1.2687 - val_accuracy: 0.5723
Epoch 4/25
1235/1235 [==============================] - 1993s 2s/step - loss: 0.8072 - accuracy: 0.6792 - val_loss: 5.6896 - val_accuracy: 0.4025
Epoch 5/25
1235/1235 [==============================] - 1754s 1s/step - loss: 0.7794 - accuracy: 0.6888 - val_loss: 1.6436 - val_accuracy: 0.5212
Epoch 6/25
1235/1235 [==============================] - 1677s 1s/step - loss: 0.7834 - accuracy: 0.6895 - val_loss: 1.0845 - val_accuracy: 0.6813
Epoch 7/25
1235/1235 [==============================] - 1814s 1s/step - loss: 0.7401 - accuracy: 0.7092 - val_loss: 0.9116 -

KeyboardInterrupt: 

In [ ]:

plt.style.use("ggplot")
plt.figure()
plt.plot(history.history['accuracy'],'r',label='Testing accuracy',color='green')
plt.plot(history.history['val_accuracy'],label='validation accuracy')
plt.xlabel('# Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("models/akex_acc.png")
plt.show()


plt.style.use("ggplot")
plt.figure()
plt.plot(history.history['loss'],'r',label='Testing accuracy',color='green')
plt.plot(history.history['val_loss'],label='validation accuracy')
plt.xlabel('# Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("models/alex_loss.png")
plt.show()


acc=history.history['accuracy'][-1]
print(acc)
